In [3]:
import elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import networkx as nx
#import pandas as pd
#print(pd.__version__)
#print(pd.__file__)
#import ray
#ray.shutdown()
#ray.init()
#object_store_memory = int(0.6 * ray.utils.get_system_memory() // 10 ** 9 * 10 ** 9)
#ray.init(
#            include_webui=False,
#            ignore_reinit_error=True,
#            plasma_directory="/mnt/sdb/modin",
#            object_store_memory=object_store_memory,
#        )
#ray.worker.get_address_info_from_redis_helper("<head-node-ip>:6379", "<head-node-ip>")
#import modin.pandas as pd
import numpy as np
import time
#import seaborn as sns
#from multiprocessing import Pool

es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")
res = Search(using=es, index="packetbeat*")\
        .query('bool', filter=Q('exists', field='ip')\
               & Q('exists', field='client_ip')\
               & Q('exists', field='type') & Q('exists', field='status')\
               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')\
               & Q('match', type='ICMP')
              )

response = res.execute()

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

#tqdm.pandas()



GET http://149.165.170.209:9200/packetbeat*/_search [status:200 request:8.236s]


Filtered logs: 119442
Total number of logs: 119442


In [ ]:
start_time = time.time()
df2 = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']) for hit in res),\
                    columns=['Timestamp','type','status','ip','client_ip','bytes_in','bytes_out'])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
df2[:10]

In [ ]:
ray.shutdown()